In [1]:
DATA_NAME = 'syntheticMRI2D-axial' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_horizontal_wavelet_axial_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,3,4,5,6,7,8,9
3,5155081.35,-63.98,15.13,-12.35,-4.32,0.29,0.39
4,-63.98,2791508.73,-25.97,-4.10,1.20,0.47,-0.05
5,15.13,-25.97,544153.91,-6.78,1.90,1.40,0.04
6,-12.35,-4.10,-6.78,188894.43,0.59,-0.46,0.02
7,-4.32,1.20,1.90,0.59,27444.60,-0.01,0.01
8,0.29,0.47,1.40,-0.46,-0.01,1837.60,0.01
9,0.39,-0.05,0.04,0.02,0.01,0.01,78.41


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

,3,4,5,6,7,8,9
3,1.00000,-0.00002,0.00001,-0.00001,-0.00001,0.00000,0.00002
4,-0.00002,1.00000,-0.00002,-0.00001,0.00000,0.00001,-0.00000
5,0.00001,-0.00002,1.00000,-0.00002,0.00002,0.00004,0.00001
6,-0.00001,-0.00001,-0.00002,1.00000,0.00001,-0.00002,0.00001
7,-0.00001,0.00000,0.00002,0.00001,1.00000,-0.00000,0.00001
8,0.00000,0.00001,0.00004,-0.00002,-0.00000,1.00000,0.00001
9,0.00002,-0.00000,0.00001,0.00001,0.00001,0.00001,1.00000


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

102.36069002657453

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.84062883e+12 1.07226961e+12 3.97466467e+10 4.43212427e+09
 8.35606676e+07 2.81585090e+05 5.34636556e-28]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.992081,-0.124414,-0.016252,-0.005581,-0.000810,-0.000054,-0.000002
1,0.123534,0.991313,-0.042795,-0.014249,-0.002059,-0.000138,-0.000006
2,0.020603,0.038914,0.994130,-0.098078,-0.012150,-0.000808,-0.000035
3,0.009287,0.017190,0.096560,0.993833,-0.050781,-0.003307,-0.000141
4,0.001778,0.003281,0.016857,0.049174,0.998068,-0.033797,-0.001432
5,0.000178,0.000328,0.001682,0.004856,0.033482,0.998513,-0.042700
6,0.000015,0.000028,0.000144,0.000415,0.002854,0.042626,0.999087


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.00791869254754829,
 0.00868747162442085,
 0.005870475263109576,
 0.006166705953795759,
 0.0019320846583177342,
 0.0014865271282042958,
 0.000913090501492575]